In [29]:
import sqlite3
import pandas as pd
import numpy as np
import requests
from db import call_db

In [30]:
conn = sqlite3.connect('MatchDB.db')
df = pd.read_sql_query("SELECT * FROM Games WHERE League = 'Allsvenskan' AND Country = 'SWE'", conn)

In [31]:
df.to_csv("Allsvenskan_Played_Games.csv", index=False)

In [32]:
url_list = ["https://www.fotmob.com/api/leagues?id=67"] 

start_game_url = "https://www.fotmob.com/api/matchDetails?matchId="

In [33]:
rem_games = []

In [34]:
for link in url_list:
    res = requests.get(link)
    jasondata = res.json()

    for match in jasondata['matches']['allMatches']:
        try:
            game_id = match['id']
            finished = match['status']['finished']
            cancelled = match['status']['cancelled']

            query = f"SELECT * FROM Games WHERE match_id = {game_id}"
            in_database = call_db(query)

            if not finished and not cancelled and not in_database:
                rem_games.append(game_id)

        except KeyError:
            pass

In [35]:
for game_id in rem_games:
    try:
        try:
            game_url = start_game_url + str(game_id)
            fot_res = requests.get(game_url)
            jasondata = fot_res.json()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching game data for ID {game_id}: {e}")

        match_id = jasondata['general']['matchId']
        date = jasondata['general']['matchTimeUTCDate'].split("T")[0]
        HomeTeam = jasondata['general']['homeTeam']['name'].replace("'"," ")
        AwayTeam = jasondata['general']['awayTeam']['name'].replace("'"," ")

        query = f"""
                INSERT INTO Rem_Games (match_id, date, HomeTeam, AwayTeam)
                VALUES ({match_id}, '{date}', '{HomeTeam}', '{AwayTeam}')
                """
        
        call_db(query)

    except:
        pass

    

In [36]:
conn = sqlite3.connect('MatchDB.db')
df2 = pd.read_sql_query("SELECT * FROM Rem_Games", conn)

In [37]:
df2.to_csv("Allsvenskan_Rem_Games.csv", index=False)